## Importing libraries

In [1]:
import glob
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

## Functions

In [2]:
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

def split_into_batches(df, batch_size=100000):
    nb_rows = len(df.index)
    intervals = []
  
    for i in range(0, nb_rows + 1, batch_size):
        intervals.append(i)
  
    if(intervals[-1] != nb_rows):
        intervals.append(nb_rows)
  
    batches_X = []
    batches_y = []
  
    for i in range(0, len(intervals) - 1):
        batches_X.append(train.iloc[intervals[i]:intervals[i + 1], :][features])
        batches_y.append(train.iloc[intervals[i]:intervals[i + 1], :]['good_move'])

    return batches_X, batches_y

## Data

In [3]:
path_fischer = "Data/CSV_FISCHER"
path_morphy = "Data/CSV_MORPHY"
path_capablanca = "Data/CSV_CAPABLANCA"

files_fischer = glob.glob(path_fischer + "/*.csv")
#files_morphy = glob.glob(path_morphy + "/*.csv")
#files_capablanca = glob.glob(path_capablanca + "/*.csv")
         
li = []

for filename in files_fischer:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

train = pd.concat(li, axis=0, ignore_index=True)
train = shuffle(train)
print(train.shape)
print(train.head)
features = list(train.iloc[:, 0:192].columns)
X = train[features]
y = train["good_move"]

categorical_columns = list(X.iloc[:, 0:63].columns)
numerical_columns = list(X.iloc[:, 64:192].columns)
feature_columns = []

for feature_name in categorical_columns:
    vocabulary = X[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in numerical_columns:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype = tf.float32))

batches_X, batches_y = split_into_batches(train)

(830376, 193)
<bound method NDFrame.head of           a1    b1    c1    d1    e1    f1    g1    h1    a2    b2  ... to_h7  \
632368  None  None  None  None  None  None  None  None     P     r  ...   0.0   
782177     R  None     B     Q     K     B     N     R     P     P  ...   0.0   
781003     R  None     B     Q     K     B  None     R     P     P  ...   0.0   
378446  None  None  None  None  None  None     K  None     P  None  ...   0.0   
33954   None  None  None  None  None  None  None  None     P  None  ...   0.0   
...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
182665     R     N     B     Q     K     B     N     R     P     P  ...   0.0   
470058     R     N     B     Q     K  None  None     R     P     P  ...   0.0   
727414     R  None     B     Q     K     B     N     R     P     P  ...   0.0   
19998   None  None     K     R  None  None     R  None     P     P  ...   0.0   
740845  None  None  None  None  None  None  None  None  None  Non

## Model

In [4]:
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns)
input_functions = []

for df_X, df_y in zip(batches_X, batches_y):
    input_functions.append(make_input_fn(df_X, df_y))

# train the model on all the input functions
i = 1
for input_function in input_functions:
    print('Batch: ' + str(i))
    i = i + 1
    linear_est.train(input_function)

# save the model
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(tf.feature_column.make_parse_example_spec(feature_columns))
estimator_path = linear_est.export_saved_model(r"C:\Users\giaco\OneDrive\Desktop\Informatica\Master\Codes, Projects and Researches\Chess", serving_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\giaco\\AppData\\Local\\Temp\\tmpz38jn3ub', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Batch: 1
Instructions for updating:
Use Variab

C:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\giaco\AppData\Local\Temp\tmpz38jn3ub\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 31.3035
INFO:tensorflow:loss = 0.1111041, step = 100 (3.198 sec)
INFO:tensorflow:global_step/sec: 146.648
INFO:tensorflow:loss = 0.018333005, step = 200 (0.682 sec)
INFO:tensorflow:global_step/sec: 156.087
INFO:tensorflow:loss = 0.13823853, step = 300 (0.639 sec)
INFO:tensorflow:global_step/sec: 144.748
INFO:tensorflow:loss = 0.1456427, step = 400 

C:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\giaco\AppData\Local\Temp\tmpz38jn3ub\model.ckpt-31250
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 31250...
INFO:tensorflow:Saving checkpoints for 31250 into C:\Users\giaco\AppData\Local\Temp\tmpz38jn3ub\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 31250...
INFO:tensorflow:loss = 0.25943673, step = 31250
INFO:tensorflow:global_step/sec: 33.9879
INFO:tensorflow:loss = 0.031354778, step = 31350 (2.943 sec)
INFO:tensorflow:global_step/sec: 156.844
INFO:tensorflow:loss = 0.13185452, step = 31450 (0.639 sec)
INFO:tensorflow:global_step/sec: 153.423
INFO:tensorflow:loss = 0.030681085, step = 31550 (0.652 sec)
INFO

...

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\giaco\AppData\Local\Temp\tmpz38jn3ub\model.ckpt-187500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 187500...
INFO:tensorflow:Saving checkpoints for 187500 into C:\Users\giaco\AppData\Local\Temp\tmpz38jn3ub\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 187500...
INFO:tensorflow:loss = 0.02560031, step = 187500
INFO:tensorflow:global_step/sec: 31.4203
INFO:tensorflow:loss = 0.11683848, step = 187600 (3.183 sec)
INFO:tensorflow:global_step/sec: 128.399
INFO:tensorflow:loss = 0.13973066, step = 187700 (0.780 sec)
INFO:tensorflow:global_step/sec: 133.328
INFO:tensorflow:loss = 0.13258581, step = 187800 (0.749 sec)
INFO:tensorflow:global_step/sec: 125.227
INFO:tensorflow:loss = 0.02

C:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\giaco\AppData\Local\Temp\tmpz38jn3ub\model.ckpt-218750
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 218750...
INFO:tensorflow:Saving checkpoints for 218750 into C:\Users\giaco\AppData\Local\Temp\tmpz38jn3ub\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 218750...
INFO:tensorflow:loss = 0.16143715, step = 218750
INFO:tensorflow:global_step/sec: 30.6921
INFO:tensorflow:loss = 0.027674919, step = 218850 (3.258 sec)
INFO:tensorflow:global_step/sec: 140.693
INFO:tensorflow:loss = 0.15953934, step = 218950 (0.711 sec)
INFO:tensorflow:global_step/sec: 139.126
INFO:tensorflow:loss = 0.13616964, step = 219050 (0.719 sec)
INFO:tensorflow:global_step/sec: 139.135
INFO:tensorflow:loss = 0.0

C:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\giaco\AppData\Local\Temp\tmpz38jn3ub\model.ckpt-250000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 250000...
INFO:tensorflow:Saving checkpoints for 250000 into C:\Users\giaco\AppData\Local\Temp\tmpz38jn3ub\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 250000...
INFO:tensorflow:loss = 0.030743644, step = 250000
INFO:tensorflow:global_step/sec: 32.8656
INFO:tensorflow:loss = 0.037931032, step = 250100 (3.044 sec)
INFO:tensorflow:global_step/sec: 129.034
INFO:tensorflow:loss = 0.14133173, step = 250200 (0.774 sec)
INFO:tensorflow:global_step/sec: 127.584
INFO:tensorflow:loss = 0.023853075, step = 250300 (0.783 sec)
INFO:tensorflow:global_step/sec: 132.6
INFO:tensorflow:loss = 0.1